In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import zipfile


In [ ]:
# Setup paths for local execution
import os
import sys

# Project directories
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')
SRC_DIR = os.path.join(BASE_DIR, 'src')

# Add src to path
sys.path.insert(0, SRC_DIR)


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [16]:
original = pd.read_csv('../../data/stock_data_week.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
original.head()

,Unnamed: 0,StockCode,InvoiceDate,Quantity,cumsum,cumdist,ds
0,0,10002,2009-01-12,12.0,12.0,0.0,2009-01-12
1,1,10002,2009-01-13,0.0,12.0,1.0,2009-01-13
2,2,10002,2009-01-14,0.0,12.0,2.0,2009-01-14
3,3,10002,2009-01-15,0.0,12.0,3.0,2009-01-15
4,4,10002,2009-01-16,0.0,12.0,4.0,2009-01-16


In [18]:
original.drop(['Unnamed: 0'],axis=1,inplace=True)

In [19]:
original['week_start'] = pd.to_datetime(original['ds'])

In [20]:
original['week_start'] = original['week_start'].dt.to_period('W-Sun').dt.to_timestamp()

In [21]:
original.head()

,StockCode,InvoiceDate,Quantity,cumsum,cumdist,ds,week_start
0,10002,2009-01-12,12.0,12.0,0.0,2009-01-12,2009-01-12
1,10002,2009-01-13,0.0,12.0,1.0,2009-01-13,2009-01-12
2,10002,2009-01-14,0.0,12.0,2.0,2009-01-14,2009-01-12
3,10002,2009-01-15,0.0,12.0,3.0,2009-01-15,2009-01-12
4,10002,2009-01-16,0.0,12.0,4.0,2009-01-16,2009-01-12


In [22]:
original = original.groupby(['StockCode','week_start'])['Quantity'].sum().reset_index()

In [23]:
original.head()

,StockCode,week_start,Quantity
0,20677,2010-05-31,96.000000
1,20677,2010-06-07,10.500000
2,20677,2010-06-14,32.000000
3,20677,2010-06-21,30.833333
4,20677,2010-06-28,123.666667


In [24]:
original.head()

,StockCode,week_start,Quantity
0,20677,2010-05-31,96.000000
1,20677,2010-06-07,10.500000
2,20677,2010-06-14,32.000000
3,20677,2010-06-21,30.833333
4,20677,2010-06-28,123.666667


In [25]:
price = pd.read_csv('../../data/price2.csv')

In [26]:
price.head()

,Unnamed: 0,StockCode,Price
0,0,10002,0.975348
1,1,10002R,5.133333
2,2,10080,0.536364
3,3,10109,0.420000
4,4,10120,0.243846


In [27]:
# price['StockCode'].fillna(-1,inplace=True)

In [28]:
price = price.groupby(['StockCode'])['Price'].mean().reset_index()

In [29]:
price.head()

,StockCode,Price
0,10002,0.975348
1,10002R,5.133333
2,10080,0.536364
3,10109,0.420000
4,10120,0.243846


In [30]:
price['Price'] = price['Price'].fillna(-1)

In [31]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5264 entries, 0 to 5263
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   StockCode  5264 non-null   object 
 1   Price      5264 non-null   float64
dtypes: float64(1), object(1)
memory usage: 82.4+ KB


In [32]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400718 entries, 0 to 400717
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   StockCode   400718 non-null  object        
 1   week_start  400718 non-null  datetime64[ns]
 2   Quantity    400718 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 9.2+ MB


In [33]:
price['StockCode'] = price['StockCode'].astype(str).str.strip()
original['StockCode'] = original['StockCode'].astype(str).str.strip()

In [34]:
original1=original.merge(price,on=['StockCode'],how='left').reset_index()

In [35]:
original1.head()

,index,StockCode,week_start,Quantity,Price
0,0,20677,2010-05-31,96.000000,1.583599
1,1,20677,2010-06-07,10.500000,1.583599
2,2,20677,2010-06-14,32.000000,1.583599
3,3,20677,2010-06-21,30.833333,1.583599
4,4,20677,2010-06-28,123.666667,1.583599


In [36]:
# country_stock = original['id_var'].str.split("_", n = 1, expand = True)

In [37]:
# original['Country'] = country_stock[0]
# original['StockCode'] = country_stock[1]

In [38]:
stock = original1.copy()

In [39]:
stock.head()

,index,StockCode,week_start,Quantity,Price
0,0,20677,2010-05-31,96.000000,1.583599
1,1,20677,2010-06-07,10.500000,1.583599
2,2,20677,2010-06-14,32.000000,1.583599
3,3,20677,2010-06-21,30.833333,1.583599
4,4,20677,2010-06-28,123.666667,1.583599


In [40]:
holiday = pd.read_csv('../../data/cleaned_data_week.csv')

In [41]:
holiday.head()

,Unnamed: 0,id_var,Quantity,cumsum,cumdist,holiday,ds
0,0,Australia_20685,2.0,2.0,0.0,0,2010-06-07
1,1,Australia_20685,0.0,2.0,1.0,0,2010-06-08
2,2,Australia_20685,0.0,2.0,2.0,0,2010-06-09
3,3,Australia_20685,0.0,2.0,3.0,0,2010-06-10
4,4,Australia_20685,0.0,2.0,4.0,0,2010-06-11


In [42]:
country_stock = holiday['id_var'].str.split("_", n = 1, expand = True)
holiday['Country'] = country_stock[0]
holiday['StockCode'] = country_stock[1]

In [43]:
holiday['ds'] = pd.to_datetime(holiday['ds'])

In [ ]:
holiday['week_start'] = holiday['ds'].dt.to_period('W-Sun').dt.to_timestamp()

In [ ]:
holiday = holiday.groupby(['StockCode','week_start'])['holiday'].sum().reset_index()

In [ ]:
holiday.shape

In [ ]:
stock.shape

In [ ]:
stock.drop_duplicates().shape

In [ ]:
holiday.drop_duplicates().shape

In [ ]:
holiday['StockCode'] = holiday['StockCode'].astype(str)

In [ ]:
original2 = stock.merge(holiday[['StockCode','week_start','holiday']],on=['StockCode','week_start'],how='left')

In [ ]:
original2.head()

In [ ]:
original2.shape

In [ ]:
original2.drop(['index'],axis=1,inplace=True)

In [ ]:
original2.loc[original2['holiday'] < 0,'holiday']=-1

In [ ]:
original2.to_csv('../../data/stock_week.csv')